In [1]:
import pandas as pd
import numpy as np

In [2]:
bid_df=pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\bid.csv")
auction_df=pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\auction.csv")
#df= bid_df.merge(auction_df, left_on='auction_id', right_on='record_id', suffixes=('', '_auction'))
x=pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\merge.csv")
''' merge.csv is  select * from bid join auction on bid.auction_id=auction.record_id'''

' merge.csv is  select * from bid join auction on bid.auction_id=auction.record_id'

In [3]:
x = x.groupby(['buyer_id', 'make','model'])['bid_amount'].max().reset_index()

# Create a user-item matrix with buyers as rows and (make, model, variant) combinations as columns
user_item_matrix = x.pivot_table(
    index='buyer_id', 
    columns=['make', 'model'], 
    values='bid_amount', 
    aggfunc='max'
).fillna(0)
print(user_item_matrix)

make       APRILIA ASHOK LEYLAND                         BAJAJ  \
model    STORM BS6     BADA DOST Bajaj Platina 100 ES Disc BS6   
buyer_id                                                         
131            0.0           0.0                           0.0   
134            0.0           0.0                           0.0   
333            0.0           0.0                           0.0   
645            0.0           0.0                           0.0   
1276           0.0           0.0                           0.0   
...            ...           ...                           ...   
7182           0.0           0.0                           0.0   
7184           0.0           0.0                           0.0   
7188           0.0           0.0                           0.0   
7189           0.0           0.0                           0.0   
7197           0.0           0.0                           0.0   

make                                                               \
model 

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate user similarities
user_similarity = cosine_similarity(user_item_matrix)
print(user_similarity)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
user_similarity_df.head()

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


buyer_id,131,134,333,645,1276,1373,1624,1642,1761,1892,...,7155,7168,7172,7176,7177,7182,7184,7188,7189,7197
buyer_id,,,,,,,,,,,,,,,,,,,,,
131,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.169728,0.0,0.0,0.0,0.0,0.0,0.0
134,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
333,0.0,0.0,1.000000,0.999903,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
645,0.0,0.0,0.999903,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1276,0.0,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Specify the buyer_id
buyer_id = 6928  # Replace with the specific buyer_id you are interested in

# Filter the DataFrame to get rows corresponding to the specific buyer_id
buyer_data = x[x['buyer_id'] == buyer_id]

# Select only the make and model columns
buyer_make_model = buyer_data[['make', 'model']]
print(buyer_make_model)
print(buyer_make_model.shape)

              make                          model
358  ASHOK LEYLAND                      BADA DOST
359        HYUNDAI  GRAND I10 ASTA 1.2 KAPPA VTVT
360       MAHINDRA     BOLERO PIK UP CBC MS 1.3 T
361  MARUTI SUZUKI                      SWIFT VXI
362  ROYAL ENFIELD                    INTERCEPTOR
363         SUZUKI                         ACCESS
364         SUZUKI   ACCESS SPCL EDITION DISC CBS
365         SUZUKI          BURGMAN STREET 125 CC
366         SUZUKI        Burgman Street Standard
367         SUZUKI              SUZUKI ACCESS 125
368           TATA                      TIAGO 1.2
(11, 2)


In [6]:
def predict_interest(user_id, make, model, user_similarity_df, user_item_matrix):
    # Check if the user_id exists in the similarity dataframe
    if user_id not in user_similarity_df.index:
        return 0  # Return 0 if the user is not found
    
    # Extract similar users
    similar_users = user_similarity_df[user_id].drop(user_id, errors='ignore')
    similar_users = similar_users[similar_users > 0].sort_values(ascending=False)
    #print(similar_users)
    
    # Check if similar users exist
    if similar_users.empty:
        return 0  # Return 0 if no similar users
    
    # Compute weighted sum of ratings
    weighted_sum = 0
    similarity_sum = 0
    for sim_user_id, similarity in similar_users.items():
        if user_item_matrix.at[sim_user_id, (make, model)] > 0:
            weighted_sum += similarity * user_item_matrix.at[sim_user_id, (make, model)]
            similarity_sum += similarity
            #print(similarity_sum)
    
    if similarity_sum == 0:
        return 0  # Return 0 if similarity sum is zero
    
    return weighted_sum / similarity_sum


In [7]:
def recommend_auctions(user_id, user_similarity_df, user_item_matrix, num_recommendations=5):
    # Check if the user_id exists in the user_item_matrix
    if user_id not in user_item_matrix.index:
        print(f"User ID {user_id} not found in training data. Recommending based on overall popularity.")
        # Recommend based on overall popularity
        overall_popularity = user_item_matrix.mean(axis=0).sort_values(ascending=False)
        recommended_auctions = overall_popularity.head(num_recommendations).index
        recommended_auction_details = [(make, model) for (make, model) in recommended_auctions]
        return recommended_auction_details, len(recommended_auction_details)
    
    # Get the items the user has already interacted with
    user_interactions = user_item_matrix.loc[user_id]
    interacted_items = set(user_interactions[user_interactions > 0].index)
    
    predicted_interests = {}
    for (make, model) in user_item_matrix.columns:
        if (make, model) not in interacted_items:
            predicted_interests[(make, model)] = predict_interest(user_id, make, model, user_similarity_df, user_item_matrix)
    
    # Sort auctions by predicted interest
    recommended_auctions = sorted(predicted_interests.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
    recommended_auction_details = [(make, model) for (make, model), _ in recommended_auctions]
    
    return recommended_auction_details, len(recommended_auction_details)

# Example: Recommend auctions for buyer 134
buyer_id = 131
recommended_auction_details, num_recommendations = recommend_auctions(buyer_id, user_similarity_df, user_item_matrix,  num_recommendations=5)
print(f"Recommended auctions: {recommended_auction_details}")
print(f"Number of recommendations: {num_recommendations}")

# Example: Recommend auctions for a new buyer (not in user_item_matrix)
new_buyer_id = 1  
recommended_auction_details, num_recommendations = recommend_auctions(new_buyer_id, user_similarity_df, user_item_matrix, num_recommendations=5)
print(f"Recommended auctions for new buyer: {recommended_auction_details}")
print(f"Number of recommendations: {num_recommendations}")


Recommended auctions: [('MAHINDRA', 'XUV 300'), ('NEW HOLLAND', '3630 TURBO 55 TRACTOR'), ('SWARAJ', 'SWARAJ 744 XT 48HP'), ('MAHINDRA', ' 575DI MKM SARPANCH WETLAND-BS3A'), ('SONALIKA', 'SONALIKA DI 50RX OIB PS MULTISPED SHUTLE')]
Number of recommendations: 5
User ID 1 not found in training data. Recommending based on overall popularity.
Recommended auctions for new buyer: [('TATA', 'FULLY-BUILT/SIGNA 3525 HD'), ('HYUNDAI', 'R 215'), ('MAHINDRA', 'XUV 300'), ('HYUNDAI', 'Santro Sportz'), ('SWARAJ', '735')]
Number of recommendations: 5


In [11]:
# Generate recommendations for all buyers
all_recommendations = []

for buyer_id in user_item_matrix.index:
    recommended_auction_details, num_recommendations = recommend_auctions(
        buyer_id, user_similarity_df, user_item_matrix, num_recommendations=5
    )
    all_recommendations.append((buyer_id, recommended_auction_details))

# Convert the recommendations to a DataFrame
recommendations_df = pd.DataFrame(all_recommendations, columns=['buyer_id', 'recommendations'])

# Save the recommendations to an Excel sheet
recommendations_df.to_excel(r"C:\Users\Harsh\Desktop\cf.csv", index=False)


ValueError: No engine for filetype: 'csv'

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load data
bid_df = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\bid.csv")
auction_df = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\auction.csv")
x = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\merge.csv")

# Group by buyer_id, make, and model to get max bid_amount
x = x.groupby(['buyer_id', 'make', 'model'])['bid_amount'].max().reset_index()

# Create user-item matrix
user_item_matrix = x.pivot_table(
    index='buyer_id',
    columns=['make', 'model'],
    values='bid_amount',
    aggfunc='max'
).fillna(0)

# Calculate user similarities
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def predict_interest(user_id, make, model, user_similarity_df, user_item_matrix):
    if user_id not in user_similarity_df.index:
        return 0
    
    similar_users = user_similarity_df[user_id].drop(user_id, errors='ignore')
    similar_users = similar_users[similar_users > 0].sort_values(ascending=False)
    
    if similar_users.empty:
        return 0
    
    weighted_sum = 0
    similarity_sum = 0
    for sim_user_id, similarity in similar_users.items():
        if user_item_matrix.at[sim_user_id, (make, model)] > 0:
            weighted_sum += similarity * user_item_matrix.at[sim_user_id, (make, model)]
            similarity_sum += similarity
    
    if similarity_sum == 0:
        return 0
    
    return weighted_sum / similarity_sum

def recommend_auctions(user_id, user_similarity_df, user_item_matrix, num_recommendations=5):
    if user_id not in user_item_matrix.index:
        overall_popularity = user_item_matrix.mean(axis=0).sort_values(ascending=False)
        recommended_auctions = overall_popularity.head(num_recommendations).index
        return list(recommended_auctions)
    
    user_interactions = user_item_matrix.loc[user_id]
    interacted_items = set(user_interactions[user_interactions > 0].index)
    
    predicted_interests = {}
    for (make, model) in user_item_matrix.columns:
        if (make, model) not in interacted_items:
            predicted_interests[(make, model)] = predict_interest(user_id, make, model, user_similarity_df, user_item_matrix)
    
    recommended_auctions = sorted(predicted_interests.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
    return [(make, model) for (make, model), _ in recommended_auctions]

# Generate recommendations for all buyers
recommendations = []
for buyer_id in user_item_matrix.index:
    recommended_auctions = recommend_auctions(buyer_id, user_similarity_df, user_item_matrix, num_recommendations=5)
    recommendations.append([buyer_id] + [f"{make} {model}" for make, model in recommended_auctions])

# Convert recommendations to DataFrame
recommendations_df = pd.DataFrame(recommendations, columns=["buyer_id"] + [f"Recommendation {i+1}" for i in range(5)])

# Save to Excel
recommendations_df.to_excel(r"C:\Users\Harsh\Desktop\cb.xlsx", index=False)


In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load data
bid_df = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\bid.csv")
auction_df = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\auction.csv")
x = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\merge.csv")

# Group by buyer_id, make, and model to get max bid_amount
x = x.groupby(['buyer_id', 'make', 'model'])['bid_amount'].max().reset_index()

# Create user-item matrix
user_item_matrix = x.pivot_table(
    index='buyer_id',
    columns=['make', 'model'],
    values='bid_amount',
    aggfunc='max'
).fillna(0)

# Calculate user similarities
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def predict_interest(user_id, make, model, user_similarity_df, user_item_matrix):
    if user_id not in user_similarity_df.index:
        return 0
    
    similar_users = user_similarity_df[user_id].drop(user_id, errors='ignore')
    similar_users = similar_users[similar_users > 0].sort_values(ascending=False)
    
    if similar_users.empty:
        return 0
    
    weighted_sum = 0
    similarity_sum = 0
    for sim_user_id, similarity in similar_users.items():
        if user_item_matrix.at[sim_user_id, (make, model)] > 0:
            weighted_sum += similarity * user_item_matrix.at[sim_user_id, (make, model)]
            similarity_sum += similarity
    
    if similarity_sum == 0:
        return 0
    
    return weighted_sum / similarity_sum

def recommend_auctions(user_id, user_similarity_df, user_item_matrix, num_recommendations=5):
    if user_id not in user_item_matrix.index:
        overall_popularity = user_item_matrix.mean(axis=0).sort_values(ascending=False)
        recommended_auctions = overall_popularity.head(num_recommendations).index
        return list(recommended_auctions)
    
    user_interactions = user_item_matrix.loc[user_id]
    interacted_items = set(user_interactions[user_interactions > 0].index)
    
    predicted_interests = {}
    for (make, model) in user_item_matrix.columns:
        if (make, model) not in interacted_items:
            predicted_interests[(make, model)] = predict_interest(user_id, make, model, user_similarity_df, user_item_matrix)
    
    recommended_auctions = sorted(predicted_interests.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
    return [(make, model) for (make, model), _ in recommended_auctions]

# Generate recommendations for all buyers
recommendations = []
for buyer_id in user_item_matrix.index:
    user_interactions = user_item_matrix.loc[buyer_id]
    interacted_items = set(user_interactions[user_interactions > 0].index)
    previously_bid_cars = list(interacted_items)
    
    recommended_auctions = recommend_auctions(buyer_id, user_similarity_df, user_item_matrix, num_recommendations=5)
    row = [buyer_id] + [f"{make} {model}" for make, model in previously_bid_cars] + [f"{make} {model}" for make, model in recommended_auctions]
    recommendations.append(row)

# Determine the maximum number of previously bid and recommended cars
max_prev_bids = max(len(row) - 1 - 5 for row in recommendations)
max_recommendations = 5

# Create column names
columns = ["buyer_id"] + [f"Previously Bid {i+1}" for i in range(max_prev_bids)] + [f"Recommendation {i+1}" for i in range(max_recommendations)]

# Convert recommendations to DataFrame
recommendations_df = pd.DataFrame(recommendations, columns=columns)

# Save to Excel
recommendations_df.to_excel(r"C:\Users\Harsh\Desktop\cf.xlsx", index=False)


In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load data
bid_df = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\bid.csv")
auction_df = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\auction.csv")
x = pd.read_csv(r"C:\Users\Harsh\Desktop\SheerDrive\merge.csv")

# Group by buyer_id, make, and model to get max bid_amount
x = x.groupby(['buyer_id', 'make', 'model'])['bid_amount'].max().reset_index()

# Create user-item matrix
user_item_matrix = x.pivot_table(
    index='buyer_id',
    columns=['make', 'model'],
    values='bid_amount',
    aggfunc='max'
).fillna(0)

# Calculate user similarities
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def predict_interest(user_id, make, model, user_similarity_df, user_item_matrix):
    if user_id not in user_similarity_df.index:
        return 0
    
    similar_users = user_similarity_df[user_id].drop(user_id, errors='ignore')
    similar_users = similar_users[similar_users > 0].sort_values(ascending=False)
    
    if similar_users.empty:
        return 0
    
    weighted_sum = 0
    similarity_sum = 0
    for sim_user_id, similarity in similar_users.items():
        if user_item_matrix.at[sim_user_id, (make, model)] > 0:
            weighted_sum += similarity * user_item_matrix.at[sim_user_id, (make, model)]
            similarity_sum += similarity
    
    if similarity_sum == 0:
        return 0
    
    return weighted_sum / similarity_sum

def recommend_auctions(user_id, user_similarity_df, user_item_matrix, num_recommendations=5):
    if user_id not in user_item_matrix.index:
        overall_popularity = user_item_matrix.mean(axis=0).sort_values(ascending=False)
        recommended_auctions = overall_popularity.head(num_recommendations).index
        return list(recommended_auctions)
    
    user_interactions = user_item_matrix.loc[user_id]
    interacted_items = set(user_interactions[user_interactions > 0].index)
    
    predicted_interests = {}
    for (make, model) in user_item_matrix.columns:
        if (make, model) not in interacted_items:
            predicted_interests[(make, model)] = predict_interest(user_id, make, model, user_similarity_df, user_item_matrix)
    
    recommended_auctions = sorted(predicted_interests.items(), key=lambda x: x[1], reverse=True)[:num_recommendations]
    return [(make, model) for (make, model), _ in recommended_auctions]

# Generate recommendations for all buyers
recommendations = []
for buyer_id in user_item_matrix.index:
    user_interactions = user_item_matrix.loc[buyer_id]
    interacted_items = set(user_interactions[user_interactions > 0].index)
    previously_bid_cars = [(make, model) for make, model in interacted_items]
    
    recommended_auctions = recommend_auctions(buyer_id, user_similarity_df, user_item_matrix, num_recommendations=5)
    row = [buyer_id, previously_bid_cars] + [f"{make} {model}" for make, model in recommended_auctions]
    recommendations.append(row)

# Determine the maximum number of recommendations
max_recommendations = 5

# Create column names
columns = ["buyer_id", "previously_bid"] + [f"Recommendation {i+1}" for i in range(max_recommendations)]

# Convert recommendations to DataFrame
recommendations_df = pd.DataFrame(recommendations, columns=columns)

# Save to Excel
recommendations_df.to_excel(r"C:\Users\Harsh\Desktop\cf.xlsx", index=False)
